
# Welcome to xeus-ocaml! 🚀

`xeus-ocaml` is a Jupyter kernel for the OCaml programming language that runs entirely in your web browser, powered by WebAssembly.

This notebook will walk you through its key features:
- **Interactive OCaml Toplevel**: Run OCaml code and see results instantly.
- **Code Intelligence**: Get code completion and type/documentation inspection, thanks to Merlin.
- **Rich Display**: Render HTML, Markdown, plots, and more directly from OCaml.
- **Dynamic Library Loading**: Use the `#require` directive to load external libraries like `ocamlgraph`.
- **Virtual Filesystem**: Work with files using standard OCaml I/O functions.

Let's get started!

## 1. Basic Toplevel Execution

You can execute OCaml code just like in a regular toplevel. Each phrase must end with `;;`.

In [ ]:
1 + 1;;

"Hello, " ^ "Jupyter!";;

The kernel maintains its state between cells. You can define a function in one cell and use it in another.

In [ ]:
let greet name = "Hello, " ^ name ^ "!";;

In [ ]:
greet "xeus-ocaml";;


## 2. Code Intelligence with Merlin

`xeus-ocaml` integrates Merlin to provide an IDE-like experience.

#### Code Completion
In the cell below, place your cursor after `List.` and press `Tab` to see a list of available functions.


In [ ]:
List.

#### Code Inspection

You can view the type and documentation for an identifier by placing your cursor over it and pressing `Shift+Tab`. You can also open the Helper with `Ctrl+I`

In [ ]:
List.map

## 3. Rich Display with `Xlib`

The `Xlib` module is automatically opened at startup, providing functions to render rich outputs like HTML, Markdown, and even interactive plots.


In [ ]:
output_html "<h1>This is an HTML Header</h1><p>Rendered from OCaml!</p>";;

output_markdown "# And this is Markdown\n\n* With a bullet point!\n* And another.";;


You can also display structured data like JSON and create interactive plots using Vega-Lite.

In [ ]:
(* Display a JSON object *)
let json_string = "{\"name\": \"xeus-ocaml\", \"is_awesome\": true}";;
output_json json_string;;

(* Display a Vega-Lite bar chart *)
let vega_spec = {|
  {
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "description": "A simple bar chart.",
    "data": {
      "values": [
        {"a": "A", "b": 28}, {"a": "B", "b": 55}, {"a": "C", "b": 43},
        {"a": "D", "b": 91}, {"a": "E", "b": 81}, {"a": "F", "b": 53}
      ]
    },
    "mark": "bar",
    "encoding": {
      "x": {"field": "a", "type": "nominal"},
      "y": {"field": "b", "type": "quantitative"}
    }
  }
|};;
output_vegalite vega_spec;;


## 4. Dynamic Library Loading with `ocamlgraph`

You can load external libraries using the `#require` directive. Let's load `ocamlgraph` to build and visualize a graph.


In [ ]:
#require "ocamlgraph";;

### Creating a Simple Graph

Let's start by defining a module for a simple, persistent, directed graph.
- **`Persistent`**: The graph data structure is immutable. Every operation (like adding an edge) returns a *new* graph.
- **`Digraph`**: The graph is directed (edges have a source and a destination).
- **`Concrete`**: The vertices themselves are the labels (in this case, `string`).


In [ ]:
open Graph;;

module G = Persistent.Digraph.Concrete(
  struct
    type t = string
    let compare = String.compare
    let hash = Hashtbl.hash
    let equal = (=)
  end
);;

(* Create an empty graph *)
let g = G.empty;;

print_endline "Empty graph created.";;

### Adding Vertices and Edges

Since our graph is persistent, we re-assign the `g` variable with the new graph returned by each operation. We'll model a simple graph of flights between cities.

In [ ]:
let g = G.add_vertex g "Paris";;
let g = G.add_vertex g "London";;
let g = G.add_vertex g "New York";;
let g = G.add_vertex g "Tokyo";;

let g = G.add_edge g "London" "Paris";;
let g = G.add_edge g "Paris" "New York";;
let g = G.add_edge g "New York" "Tokyo";;
let g = G.add_edge g "Tokyo" "London";; (* A cycle! *)
let g = G.add_edge g "Paris" "Tokyo";;

print_endline "Added 4 vertices and 5 edges.";;

### Inspecting the Graph

We can now query the graph for basic information like the number of vertices and edges, and iterate over them.

In [ ]:
Printf.printf "Number of vertices: %d\n" (G.nb_vertex g);;
Printf.printf "Number of edges: %d\n" (G.nb_edges g);;

print_endline "\nVertices:";;
G.iter_vertex (fun v -> print_endline v) g;;

print_endline "\nEdges:";;
G.iter_edges_e (fun (src, dst) -> Printf.printf "%s -> %s\n" src dst) g;;

### Running a Simple Algorithm: Depth-First Search (DFS)

OCamlGraph comes with many pre-built algorithms. Let's use a Depth-First Search traversal. We create a new module `Dfs` by applying the `Graph.Traverse.Dfs` functor to our graph module `G`.

In [ ]:
module Dfs = Traverse.Dfs(G);;

#### DFS traversal order (prefix):

In [ ]:
Dfs.iter ~pre:(fun v -> Printf.printf "%s " v) g;;

#### Check for cycles

In [ ]:
if Dfs.has_cycle g then
  print_endline "\nThe graph has at least one cycle."
else
  print_endline "\nThe graph is a DAG (Directed Acyclic Graph).";;

### Visualization with Graphviz (Dot format)

OCamlGraph can generate output in the **DOT language**, which can be used by the Graphviz tool suite to render an image of the graph.

First, we create a `Dot` module configured for our graph `G`, specifying how vertices and the overall graph should be styled.

In [ ]:
module Dot = Graphviz.Dot(
  struct
    include G (* Use our graph module `G` *)

    (* Graph-level attributes *)
    let graph_attributes _ = [`Rankdir `LeftToRight]
    
    (* Default vertex attributes *)
    let default_vertex_attributes _ = []
    
    (* Vertex-specific attributes *)
    let vertex_name v = Printf.sprintf "\"%s\"" v (* Quote vertex names *)
    let vertex_attributes _ = [`Shape `Box; `Style `Rounded]
    
    (* Default edge attributes *)
    let default_edge_attributes _ = []
    
    (* Edge-specific attributes *)
    let edge_attributes _ = []
    
    (* Subgraph handling (not used here) *)
    let get_subgraph _ = None
  end
);;

### Generating the Graph dot string (graphviz)

Now, let's generate the DOT representation and print it !

In [ ]:
(* Generate the DOT output to a string buffer *)
let dot_string = 
    let buffer = Buffer.create 1024 in
    let formatter = Format.formatter_of_buffer buffer in
    Dot.fprint_graph formatter g;
    Format.pp_print_flush formatter ();
    Buffer.contents buffer

### SVG Visualization with builtin extension and viz.js

In [ ]:
output_dot dot_string

## 5. Virtual Filesystem

The kernel includes an in-memory virtual filesystem. You can use standard OCaml `In_channel`, `Out_channel`, and `Sys` functions to interact with it.

In [ ]:
(* Write a message to a file *)
let message = "Hello from the virtual filesystem!";;
let oc = open_out "hello.txt";;
output_string oc message;;
close_out oc;;

In [ ]:
(* Read the message back *)
let ic = open_in "hello.txt";;
let read_message = input_line ic;;
close_in ic;;

read_message;;

We can verify the file was created by listing the contents of the current directory.

In [ ]:
Sys.readdir ".";;

## 🎉 Conclusion

This notebook has demonstrated the core features of `xeus-ocaml`. You've seen how to execute code, get rich language support, display visualizations, load libraries, and interact with a virtual filesystem, all within your browser.

For more information, visit the [project repository on GitHub](https://github.com/davy39/xeus-ocaml).